In [73]:
import json
import numpy as np

In [74]:
with open("content.json", 'r') as f:
    data = json.load(f)
# data

In [75]:
paper_to_index = {}
for index in range(len(data)):
    paper_to_index[data[index]["id"]] = index
# paper_to_index

In [76]:
adj_matrice = np.zeros((len(data), len(data)))
for i in range(len(data)):
    refs = data[i]["references"]
    for ref in refs:
        j = paper_to_index.get(ref, -1)
        if j != -1:
            adj_matrice[i, j] = 1     

In [79]:
def cal_pageRank(adj_matrice, alpha):
    P = adj_matrice.copy()
    V = np.zeros(len(P[0]))
    V.fill(1 / len(P[0]))
    
    for i in range(len(P)):
        if sum(P[i]) == 0:
            P[i] = V
        else:
            P[i] = (1 - alpha) * P[i] + alpha * V
    res = V.reshape((1, len(V))).dot(P)
    for i in range(10):
        res = res.dot(P)
    
    return dict([(i, res[0, paper_to_index[i]]) for i in paper_to_index])

In [81]:
P = cal_pageRank(adj_matrice, 0.5)
# P

In [82]:
data

[{'id': '2981549002',
  'title': 'Tensor Programs I: Wide Feedforward or Recurrent Neural Networks of Any Architecture are Gaussian Processes.',
  'abstract': 'Wide neural networks with random weights and biases are Gaussian processes, as originally observed by Neal (1995) and more recently by Lee et al. (2018) and Matthews et al. (2018) for deep fully-connected networks, as well as by Novak et al. (2019) and Garriga-Alonso et al. (2019) for deep convolutional networks. We show that this Neural Network-Gaussian Process correspondence surprisingly extends to all modern feedforward or recurrent neural networks composed of multilayer perceptron, RNNs (e.g. LSTMs, GRUs), (nD or graph) convolution, pooling, skip connection, attention, batch normalization, and/or layer normalization. More generally, we introduce a language for expressing neural network computations, and our result encompasses all such expressible neural networks. This work serves as a tutorial on the *tensor programs* techni

In [92]:
authors_to_index = {}
for paper in data:
    for author in paper["authors"]:
        if author not in authors_to_index:
            authors_to_index[author] = len(authors_to_index)
authors_to_index

{'Greg Yang': 0,
 'Linda Wang': 1,
 'Zhong Qiu Lin': 2,
 'Alexander Wong': 3,
 'Han Zhang': 4,
 'Ian Goodfellow': 5,
 'Dimitris Metaxas': 6,
 'Augustus Odena': 7,
 'Alexey Dosovitskiy': 8,
 'Lucas Beyer': 9,
 'Alexander Kolesnikov': 10,
 'Dirk Weissenborn': 11,
 'Xiaohua Zhai': 12,
 'Thomas Unterthiner': 13,
 'Mostafa Dehghani': 14,
 'Matthias Minderer': 15,
 'Georg Heigold': 16,
 'Sylvain Gelly': 17,
 'Jakob Uszkoreit': 18,
 'Neil Houlsby': 19,
 'Volodymyr Mnih': 20,
 'Koray Kavukcuoglu': 21,
 'David Silver': 22,
 'Andrei A. Rusu': 23,
 'Joel Veness': 24,
 'Marc G. Bellemare': 25,
 'Alex Graves': 26,
 'Martin Riedmiller': 27,
 'Andreas K. Fidjeland': 28,
 'Georg Ostrovski': 29,
 'Stig Petersen': 30,
 'Charles Beattie': 31,
 'Amir Sadik': 32,
 'Ioannis Antonoglou': 33,
 'Helen King': 34,
 'Dharshan Kumaran': 35,
 'Daan Wierstra': 36,
 'Shane Legg': 37,
 'Demis Hassabis': 38,
 'Tomas Mikolov': 39,
 'Ilya Sutskever': 40,
 'Kai Chen': 41,
 'Greg S Corrado': 42,
 'Jeff Dean': 43,
 'Kaiming

In [95]:
authors_ref_matrix = np.zeros((len(authors_to_index), len(authors_to_index)))
for i in range(len(data)):
    authors = data[i]["authors"]
    refs = data[i]["references"]
    for j in range(len(data)):
        if i != j:
            if data[j]["id"] in refs:
                target_authors = data[j]["authors"]
                for a in authors:
                    for b in target_authors:
                        authors_ref_matrix[authors_to_index[a], authors_to_index[b]] = 1

In [119]:
h = np.ones(len(authors_ref_matrix))
a = np.ones(len(authors_ref_matrix))
for rep in range(5):
    for i in range(len(authors_ref_matrix)):
        h[i] += a.reshape((1, len(a))).dot(authors_ref_matrix[i].reshape((len(authors_ref_matrix), 1)))
        
    for i in range(len(authors_ref_matrix)):
        a[i] += h.reshape((1, len(h))).dot(authors_ref_matrix[:, i])
    a = a / sum(a)
    h = h / sum(h)
    

In [130]:
authorities = dict([(i, a[authors_to_index[i]]) for i in authors_to_index])
sorted(authorities.items(), key = lambda x: x[1], reverse = True)

[('Christian Drosten', 0.003954036439205177),
 ('', 0.003726857425419863),
 ('Stephan Günther', 0.0033668895591079493),
 ('Ksiazek Tg', 0.003045150427858884),
 ('Erdman D', 0.003045150427858884),
 ('Goldsmith Cs', 0.003045150427858884),
 ('Zaki', 0.003045150427858884),
 ('Peret T', 0.003045150427858884),
 ('Emery S', 0.003045150427858884),
 ('Tong S', 0.003045150427858884),
 ('Urbani C', 0.003045150427858884),
 ('Comer Ja', 0.003045150427858884),
 ('Lim W', 0.003045150427858884),
 ('Rollin Pe', 0.003045150427858884),
 ('Dowell Sf', 0.003045150427858884),
 ('Ling Ae', 0.003045150427858884),
 ('Humphrey Cd', 0.003045150427858884),
 ('Shieh Wj', 0.003045150427858884),
 ('Guarner J', 0.003045150427858884),
 ('Paddock Cd', 0.003045150427858884),
 ('Rota P', 0.003045150427858884),
 ('Fields B', 0.003045150427858884),
 ('DeRisi J', 0.003045150427858884),
 ('Yang Jy', 0.003045150427858884),
 ('Cox N', 0.003045150427858884),
 ('Hughes Jm', 0.003045150427858884),
 ('LeDuc Jw', 0.0030451504278588

In [129]:
for paper in data:
    if '' in paper["authors"]:
        print(paper["authors"])

['Kyunghyun Cho', 'Bart van Merrienboer', 'Caglar Gulcehre', 'Dzmitry Bahdanau', 'Fethi Bougares', 'Holger Schwenk', 'Yoshua Bengio', '', '']
['Yann Lecun', 'Leon Bottou', '', 'Yoshua Bengio', '', '', 'Patrick Haffner', '']
['Yann LeCun', '', 'Yoshua Bengio', 'Geoffrey Hinton', '']
['Kyunghyun Cho', 'Bart van Merrienboer', 'Dzmitry Bahdanau', 'Yoshua Bengio', '', '']
['Abdullah Assiri', 'Jaffar A Al-Tawfiq', 'Abdullah A Al-Rabeeah', 'Fahad A Al-Rabiah', 'Sami Al-Hajjar', 'Ali Al-Barrak', 'Hesham Flemban', 'Wafa N Al-Nassir', 'Hanan H Balkhy', 'Rafat F Al-Hakeem', 'Hatem Q Makhdoom', 'Alimuddin I Zumla', '', 'Ziad A Memish']
['Na Zhu', 'Dingyu Zhang', 'Wenling Wang', 'Xingwang Li', 'Bo Yang', 'Jingdong Song', 'Xiang Zhao', 'Baoying Huang', 'Weifeng Shi', 'Roujian Lu', 'Peihua Niu', 'Faxian Zhan', 'Xuejun Ma', '', 'Dayan Wang', 'Wenbo Xu', 'Guizhen Wu', 'George F. Gao', 'Wenjie Tan']
['Jasper Fuk Woo Chan', 'Shuofeng Yuan', 'Kin Hang Kok', 'Kelvin Kai Wang To', '', 'Hin Chu', 'Jin Yang',